In [1]:
!pip install --upgrade pip
import re
import xml.sax.saxutils as saxutils

#from BeautifulSoup import BeautifulSoup
!pip install gensim
from gensim.models.word2vec import Word2Vec

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers import Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPooling2D
from multiprocessing import cpu_count
!pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
!pip install pandas
import numpy as np
import pandas as pd

from pandas import DataFrame
!pip install sklearn
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import cross_validation

from sklearn import metrics
from numpy import linalg
from math import sqrt

Requirement already up-to-date: pip in /opt/conda/lib/python3.5/site-packages


Using TensorFlow backend.
Slow version of gensim.models.doc2vec is being used


/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('nfr_tf-idf_matrix.csv')
feature_names = [c for c in df.columns if c not in ["class"]]
X = df[feature_names] 
print(X.shape[0])
Y = df["class"]

cols_to_transform = [ 'class' ]
df_with_dummies = pd.get_dummies(Y,columns = cols_to_transform)
#print(df_with_dummies)
# Word2Vec number of features
num_features = 300
# Limit each newsline to a fixed number of words
document_max_num_words = 100

num_categories = 12

625


In [3]:
from keras.callbacks import Callback

class BatchLogger(Callback):
    def on_train_begin(self, epoch, logs={}):
        self.log_values = {}
        for k in self.params['metrics']:
            self.log_values[k] = []

    def on_epoch_end(self, batch, logs={}):
        for k in self.params['metrics']:
            if k in logs:
                self.log_values[k].append(logs[k])
    
    def get_values(self, metric_name, window):
        d =  pd.Series(self.log_values[metric_name])
        return d.rolling(window,center=False).mean()

bl = BatchLogger()

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import KFold
#X_train, X_test, Y_train, Y_test = train_test_split(X, df_with_dummies, test_size=0.3)

df_label_NM = df_with_dummies.as_matrix()
#df_label_NM_1D = df_label_NM.ravel()
df_data_NM = X.as_matrix()

#skf = cross_validation.StratifiedKFold(df_label_NM,n_folds=10)
skf = StratifiedKFold(n_splits=10)
kf = KFold(X.shape[0], n_folds=10)


fold_score = []
fold_accuracy = []
fold_mean = []
fold_number = 1

#for train_index, test_index in skf:
#for train_index, test_index in skf.split(df_data_NM, df_label_NM):
for train_index, test_index in kf:
    #print("Fold Number:",fold_number)
    fold_number+=1
    x_train, x_test = df_data_NM[train_index], df_data_NM[test_index]
    y_train, y_test = df_label_NM[train_index], df_label_NM[test_index]
    
    input_dim = x_train.shape[1]

    model = Sequential()
    model.add(Dense(256, input_shape=(input_dim,), activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='sigmoid'))
    model.add(Dense(12,  activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    history = model.fit(
              np.array(x_train), np.array(y_train),
              batch_size=25, nb_epoch=100, verbose=1, callbacks=[bl],
              validation_data=(np.array(x_test), np.array(y_test)))
    
    score = model.evaluate(np.array(x_test), np.array(y_test), verbose=0)
    print('Test log loss:', score[0])
    print('Test accuracy:', score[1])
    
    score_accracy = score[1]
    fold_accuracy.append(score_accracy)
    
print("Accuracy per fold: ", fold_accuracy)
print("Average Accuracy: ", sum(fold_accuracy)/len(fold_accuracy), "\n")
    
    

#x = X_test.reshape((12,437,300))
#y = Y_test.reshape((12,437,300))

Train on 562 samples, validate on 63 samples
Epoch 1/100
562/562 [==============================] - 0s - loss: 0.7721 - acc: 0.3176 - val_loss: 0.7141 - val_acc: 0.4550
Epoch 2/100
562/562 [==============================] - 0s - loss: 0.6790 - acc: 0.4678 - val_loss: 0.6604 - val_acc: 0.5278